# 檢索問答

## Step1 下載FAQ數據

In [15]:
!pip install transformers[torch]
!pip install datasets
!pip install evaluate
!pip install trainer

In [16]:
!pip install faiss-cpu

In [17]:
# from google.colab import drive
# drive.mount('/content/drive')

In [18]:
import pandas as pd

data = pd.read_csv("./law_faq.csv")
data.head()


,title,reply
0,在法律中定金与订金的区别订金和定金哪个受,“定金”是指当事人约定由一方向对方给付的，作为债权担保的一定数额的货币，它属于一种法律上的担...
1,盗窃罪的犯罪客体是什么，盗窃罪的犯罪主体,盗窃罪的客体要件本罪侵犯的客体是公私财物的所有权。侵犯的对象，是国家、集体或个人的财物，一般...
2,非法微整形机构构成非法经营罪吗,符合要件就有可能。非法经营罪，是指未经许可经营专营、专卖物品或其他限制买卖的物品，买卖进出口...
3,入室持刀行凶伤人能不能判刑,对于入室持刀伤人涉嫌故意伤害刑事犯罪，一经定罪，故意伤害他人身体的，处三年以下有期徒刑、拘役...
4,对交通事故责任认定书不服怎么办，交通事故损,事故认定书下发后，如果你对认定不满意，可在接到认定书3日内到上一级公安机关复议。


In [19]:
data["title"][0]

'在法律中定金与订金的区别订金和定金哪个受'

## Step2 下載模型

In [20]:
!pwd

'pwd' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC


In [21]:
import sys
sys.path.append('D:\python_workspace\iSpan_LLM-NLP-cookbooks\HuggingFace_scratch\sunny_huginfs_NLP\advance_task\05-retrieval_chatbot')

In [22]:
from dual_model import DualModel

dual_model = DualModel.from_pretrained(r"D:\python_workspace\iSpan_LLM-NLP-cookbooks\HuggingFace_scratch\sunny_huginfs_NLP\advance_task\04-sentence_similarity\dual_model_cp")
dual_model = dual_model.cuda()
dual_model.eval()
print("匹配模型下載成功！")

匹配模型下載成功！


In [23]:
from transformers import AutoTokenizer
tokenzier = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")
tokenzier

BertTokenizerFast(name_or_path='hfl/chinese-macbert-base', vocab_size=21128, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

## Step3 將問題編碼成向量

In [24]:
data["title"].to_list()

['在法律中定金与订金的区别订金和定金哪个受',
 '盗窃罪的犯罪客体是什么，盗窃罪的犯罪主体',
 '非法微整形机构构成非法经营罪吗',
 '入室持刀行凶伤人能不能判刑',
 '对交通事故责任认定书不服怎么办，交通事故损',
 '喝完就又去唱歌回家时去了车祸去唱歌我同意了车是我开的',
 '二审一般要多久',
 '欠钱没有欠条有录音能起诉吗',
 '从合伙经营公司撤资，只给欠条有用吗',
 '哪些情况可以先行刑事拘留',
 '是一个撞了人的不陪偿',
 '在工地上面出了安全事故怎么来维护自己的权',
 '申请和受理劳动争议调解仲裁时间上有什么限制',
 '商标注册有什么相对禁止的条件',
 '高利贷还不还不知如何是好。',
 '公司破产孕妇怎么赔偿',
 '两个人打架,因为对方先打我,后来找着打他的最后对方叫来五六人把我的肋骨打断',
 '客户拖欠货款',
 '到法院起诉一个人，要多少费用',
 '签了购房合同等于取得房屋产权吗',
 '迷信诈骗团伙老板和骨干会被判刑多少年',
 '婚后买房，婆婆出了一半钱，离婚房怎么分',
 '刑事案件，怎么收费，请律师都是有哪些费用',
 '我是个大车司机，老板拖欠工资我用什么方法能',
 '交通事故对方主责我次责车辆修理费怎么算',
 '你好，花了别人偷来送给你的钱，在公安局画押教罚款会不会案底',
 '请问，父亲生前欠下债务要还吗',
 '起诉欠款人的可行法律流程',
 '公司拖欠工资打什么电话投诉才可以',
 '结婚二+多年没在一起算是自动离婚不',
 '钱站利息算高利贷吗？',
 '你好，在公安局的笔录上画了押，还教了罚款会不会有案底',
 '假如房产证和户口本同时给租赁人办理营业执照会发生什么后果？',
 '请问我是二婚，房子是婚前买的，上午办的结婚证，下午去加的老婆名字，这样算不算夫妻共同财产',
 '是轻伤，他现在被拘留了还是拘留几天就放了',
 '如何区分贪污和受贿？',
 '公司法人变更工程代建合同是否还有效',
 '带未满周岁孩子无经济来源的签订担保有效吗',
 '女方欠男方的钱，离婚协议书要不要写让她归还？',
 '购房合同丢了怎么办',
 '新房与效果图不一致可否退房',
 '公司有权没收手机吗？',
 '通*亚*拖*工*工*',
 '老板拖欠工资拒绝给押金怎么办',
 '您好，我现在是监外执行期间，信

In [25]:
import torch
from tqdm import tqdm
questions = data["title"].to_list()
vectors = []
with torch.inference_mode():
    for i in tqdm(range(0, len(questions), 32)): # batch 32
        batch_sens = questions[i: i + 32]
        inputs = tokenzier(batch_sens, return_tensors="pt", padding=True, max_length=128, truncation=True)
        inputs = {k: v.to(dual_model.device) for k, v in inputs.items()}
        vector = dual_model.bert(**inputs)[1]
        vectors.append(vector)
    print(vectors)
vectors = torch.concat(vectors, dim=0).cpu().numpy()
vectors.shape




100%|██████████| 570/570 [00:39<00:00, 14.31it/s]


[tensor([[-0.9560, -0.8858, -0.9980,  ...,  0.8178, -0.7231, -0.4624],
        [-0.9872, -0.9737, -0.9994,  ...,  0.8943,  0.3774, -0.9121],
        [-0.9949, -0.9947, -0.9941,  ...,  0.9753,  0.5454, -0.9657],
        ...,
        [-0.9997, -0.9999, -0.9963,  ...,  0.9960,  0.8572, -0.2648],
        [-0.9994, -0.9957, -0.9822,  ...,  0.9535,  0.0351, -0.9553],
        [-0.8488, -0.6377, -0.9291,  ..., -0.1285, -0.8756, -0.1006]],
       device='cuda:0'), tensor([[ 0.2997, -0.1604,  0.6263,  ..., -0.8748, -0.9465,  0.4468],
        [-0.9831, -0.5753, -0.9281,  ...,  0.2769, -0.5302,  0.9590],
        [-0.9705, -0.9985, -0.9743,  ...,  0.3953, -0.1157, -0.9324],
        ...,
        [-0.9932, -0.8906, -0.9968,  ...,  0.8923,  0.9108,  0.6034],
        [-0.9993, -0.9996, -0.9991,  ...,  0.9473,  0.9352, -0.8329],
        [-0.9980, -0.9988, -0.9656,  ...,  0.9960,  0.7946, -0.8316]],
       device='cuda:0'), tensor([[-0.9309, -0.9951, -0.3369,  ...,  0.7591, -0.2775, -0.7312],
        [-0

(18213, 768)

In [26]:
vectors

array([[-0.95604384, -0.8858145 , -0.9980055 , ...,  0.8178214 ,
        -0.7230785 , -0.4624097 ],
       [-0.9872227 , -0.9736988 , -0.999379  , ...,  0.8943107 ,
         0.37741658, -0.91206574],
       [-0.99492204, -0.9946514 , -0.994081  , ...,  0.97531927,
         0.5453944 , -0.96566254],
       ...,
       [-0.99886405, -0.9500373 , -0.9836558 , ...,  0.98028636,
         0.32546374, -0.74935865],
       [-0.9897317 , -0.6264473 , -0.9997929 , ...,  0.99807817,
         0.8697506 ,  0.85977805],
       [-0.99506456, -0.5374523 , -0.91540766, ...,  0.9564011 ,
        -0.05316731, -0.33935544]], dtype=float32)

## Step4 建立索引

In [27]:
import faiss

index = faiss.IndexFlatIP(768)
faiss.normalize_L2(vectors)
index.add(vectors)
index


<faiss.swigfaiss.IndexFlatIP; proxy of <Swig Object of type 'faiss::IndexFlatIP *' at 0x00000284861FF900> >

## Step5 向量編碼 -> 問題

In [28]:
quesiton = "寻衅滋事"
with torch.inference_mode():
    inputs = tokenzier(quesiton, return_tensors="pt", padding=True, max_length=128, truncation=True)
    inputs = {k: v.to(dual_model.device) for k, v in inputs.items()}
    vector = dual_model.bert(**inputs)[1]
    q_vector = vector.cpu().numpy()

q_vector.shape


(1, 768)

## Step6 向量匹配(召回)

## Vector Matching (向量匹配)
This approach involves directly comparing the vector representation of a query (e.g., a user's question) to vector representations of potential answers (e.g., FAQ entries or database records). The comparison is often made using similarity metrics such as cosine similarity, and the best match is chosen based on the highest similarity score. This method is straightforward and efficient but may not capture complex relationships or nuances between the query and the documents because it treats the vectors as independent entities.

## Vector Interaction Matching (向量交互匹配)
This method involves a more sophisticated process where the interactions between the elements of the vectors are taken into account. Instead of comparing the vectors as whole entities, the method examines how individual elements of one vector relate to individual elements of another, potentially using attention mechanisms or other forms of neural network layers that allow for a richer, more nuanced understanding of the relationship between the query and the documents. This can lead to more accurate matching because it can capture deeper semantic relationships.


In [29]:
data.values

array([['在法律中定金与订金的区别订金和定金哪个受',
        '“定金”是指当事人约定由一方向对方给付的，作为债权担保的一定数额的货币，它属于一种法律上的担保方式，目的在于促使债务人履行债务，保障债权人的债权得以实现。签合同时，对定金必需以书面形式进行约定，同时还应约定定金的数额和交付期限。给付定金一方如果不履行债务，无权要求另一方返还定金；接受定金的一方如果不履行债务，需向另一方双倍返还债务。债务人履行债务后，依照约定，定金应抵作价款或者收回。而“订金”目前我国法律没有明确规定，它不具备定金所具有的担保性质，可视为“预付款”，当合同不能履行时，除不可抗力外，应根据双方当事人的过错承担违约责任。'],
       ['盗窃罪的犯罪客体是什么，盗窃罪的犯罪主体',
        '盗窃罪的客体要件本罪侵犯的客体是公私财物的所有权。侵犯的对象，是国家、集体或个人的财物，一般是指动产而言，但不动产上之附着物，可与不动产分离的，例如，田地上的农作物，山上的树木、建筑物上之门窗等，也可以成为本罪的对象。另外，能源如电力、煤气也可成为本罪的对象。盗窃罪侵犯的客体是公私财物的所有权。所有权包括占有、使用、收益、处分等权能。这里的所有权一般指合法的所有权，但有时也有例外情况。根据《最高人民法院关于审理盗窃案件具体应用法律若干问题的解释》(以下简称《解释》)的规定：“盗窃违禁品，按盗窃罪处理的，不计数额，根据情节轻重量刑。盗窃违禁品或犯罪分子不法占有的财物也构成盗窃罪。”'],
       ['非法微整形机构构成非法经营罪吗',
        '符合要件就有可能。非法经营罪，是指未经许可经营专营、专卖物品或其他限制买卖的物品，买卖进出口许可证、进出口原产地证明以及其他法律、行政法规规定的经营许可证或者批准文件，以及从事其他非法经营活动，扰乱市场秩序，情节严重的行为。'],
       ...,
       ['老赖欠钱不还，可以用坐牢抵债吗',
        '肯定是要还的，判刑坐牢是针对他不还钱行为的惩罚，不是说坐牢抵债的，但如果一个人真的没钱你也没办法，只能看好对方的经济状况，一旦有钱的就得立马采取措施'],
       ['我男朋友把我姐夫的车给偷了，我姐夫报案了，一开始我不知道他偷车了我知道后帮他逃跑我有罪吗',
        '朋友。你男

In [30]:
faiss.normalize_L2(q_vector)
scores, indexes = index.search(q_vector, 10)
topk_result = data.values[indexes[0].tolist()]
topk_result[:, 0]

array(['涉嫌寻衅滋事', '上搞清洁跌倒是工伤吗', '打架斗殴双方都抓起来了吗', '明知道是童工还收了',
       '来，都来，咱说下什么叫道德绑架', '讨薪，讨血汗钱，我的钱就这么打水漂了吗', '蚂蚁借呗借了不还会怎么样',
       '寻衅滋事没有伤检得到对方的谅解', '高利贷不还会被拉黑', '绑架还叫什么'], dtype=object)

## Step7 加载交互模型

In [32]:
from transformers import BertForSequenceClassification
corss_model = BertForSequenceClassification.from_pretrained(r"D:\python_workspace\iSpan_LLM-NLP-cookbooks\HuggingFace_scratch\sunny_huginfs_NLP\advance_task\04-sentence_similarity\cross_model_cp")
corss_model = corss_model.cuda()
corss_model.eval()
print("模型下載成功！")



模型下載成功！


## Step8 最終預測(排序匹配)

In [37]:
topk_result[:, 0].tolist()


['涉嫌寻衅滋事',
 '上搞清洁跌倒是工伤吗',
 '打架斗殴双方都抓起来了吗',
 '明知道是童工还收了',
 '来，都来，咱说下什么叫道德绑架',
 '讨薪，讨血汗钱，我的钱就这么打水漂了吗',
 '蚂蚁借呗借了不还会怎么样',
 '寻衅滋事没有伤检得到对方的谅解',
 '高利贷不还会被拉黑',
 '绑架还叫什么']

In [33]:
canidate = topk_result[:, 0].tolist()
[quesiton] * len(canidate)

['寻衅滋事',
 '寻衅滋事',
 '寻衅滋事',
 '寻衅滋事',
 '寻衅滋事',
 '寻衅滋事',
 '寻衅滋事',
 '寻衅滋事',
 '寻衅滋事',
 '寻衅滋事']

In [34]:
canidate = topk_result[:, 0].tolist()
ques = [quesiton] * len(canidate)
inputs = tokenzier(ques, canidate, return_tensors="pt", padding=True, max_length=128, truncation=True)
inputs = {k: v.to(corss_model.device) for k, v in inputs.items()}
with torch.inference_mode():
    logits = corss_model(**inputs).logits.squeeze()
    result = torch.argmax(logits, dim=-1)
result

tensor(0, device='cuda:0')

In [35]:
canidate[0]

'涉嫌寻衅滋事'

In [38]:
canidate_answer = topk_result[:, 1].tolist()
match_quesiton = canidate[result.item()]
final_answer = canidate_answer[result.item()]
match_quesiton, final_answer



('涉嫌寻衅滋事',
 '说明具有寻衅滋事行为，应受到相应的处罚，行为人情形严重或行为恶劣的涉嫌了寻衅滋事罪。寻衅滋事是指行为人结伙斗殴的、追逐、拦截他人的、强拿硬要或者任意损毁、占用公私财物的、其他寻衅滋事的行为。寻衅滋事罪，是指在公共场所无事生非、起哄闹事，造成公共场所秩序严重混乱的，追逐、拦截、辱骂、恐吓他人，强拿硬要或者任意损毁、占用公私财物，破坏社会秩序，情节严重的行为。对于寻衅滋事行为的处罚：1、《中华人*共和国治安管理处罚法》第二十六条规定，有下列行为之一的，处五日以上十日以下拘留，可以并处五百元以下罚款;情节较重的，处十日以上十五日以下拘留，可以并处一千元以下罚款:(一)结伙斗殴的;(二)追逐、拦截他人的;(三)强拿硬要或者任意损毁、占用公私财物的;(四)其他寻衅滋事行为;2、《中华人*共和国刑法》第二百九十三条有下列寻衅滋事行为之一，破坏社会秩序的，处五年以下有期徒刑、拘役或者管制:(一)随意殴打他人，情节恶劣的;(二)追逐、拦截、辱骂、恐吓他人，情节恶劣的;(三)强拿硬要或者任意损毁、占用公私财物，情节严重的;(四)在公共场所起哄闹事。造成公共场所秩序严重混乱的。纠集他人多次实施前款行为，严重破坏社会秩序的，处五年以上十年以下有期徒刑，可以并处罚金。3、最*人*法*和最*人*检**《关于办理寻衅滋事案件的司法解释》为依法惩治寻衅滋事犯罪，维护社会秩序，最*人*法*会*最*人*检**根据《中华人*共和国刑法》的有关规定，就办理寻衅滋事刑事案件适用法律的若干问题司法解释如下:第一条行为人为寻求刺激、发泄情绪、逞强耍横等，无事生非，实施刑法第二百九十三条规定的行为的，应当认定为"寻衅滋事"。行为人因日常生活中的偶发矛盾纠纷，借故生非，实施刑法第二百九十三条规定的行为的，应当认定为"寻衅滋事"，但矛盾系由被害人故意引发或者被害人对矛盾激化负有主要责任的除外。行为人因婚恋、家庭、邻里、债务等纠纷，实施殴打、辱骂、恐吓他人或者损毁、占用他人财物等行为的，一般不认定为"寻衅滋事"，但经有关部门批评制止或者处理处罚后，继续实施前列行为，破坏社会秩序的除外。第二条随意殴打他人，破坏社会秩序，具有下列情形之一的，应当认定为刑法第二百九十三条第一款第一项规定的"情节恶劣":1、致一人以上轻伤或者二人以上轻微伤的;2、引起他人精神失常、自杀等严重后果的;3、多次随意殴打